In [80]:
import sys
sys.path.insert(0, '../src')

import pickle
from loaders import *
from episode import *
from dataset import *

from common import *
from model import Model
from meta_learner import MetaLearner

import numpy as np
import torch
from torch.autograd import Variable
import torch.utils.data.sampler as sampler

In [81]:
""" Some global variables """
_loader = Loader(502) # 500 + SOS + EOS
loader = MIDILoader(_loader)

use_cuda = True
# Is the tokenizer 1 indexed?
vocabulary_size = 16*128*2 + 32*16 + 100 + 1 # 4708 + 1
vocabulary_size = vocabulary_size + 2 # SOS (index 4709) and EOS (index 4710)
SOS_TOKEN = 4709
EOS_TOKEN = 4710

encoding_size = 500
one_hot_embeddings = np.eye(vocabulary_size)

In [82]:
lr = 0.01
meta_learner = MetaLearner(Model, (vocabulary_size,encoding_size,vocabulary_size,lr), lr, 1)

In [4]:
eps = load_sampler_from_config("../src/config.yaml")
episode = eps.get_episode()
train = episode.support
test = episode.query

Eurythmics wouldilietoyou.mid
Eurythmics dontaskmewhy.mid
Eurythmics walkingonbrokenglasses.mid
Eurythmics nomoreiloveyou.mid
Eurythmics sweetdreams.mid
Eurythmics thorninmyside.mid
Eurythmics lovesongforavampire.mid
Eurythmics lilywashere.mid
Eurythmics theremustbeanangel.mid
Eurythmics dontletitbringyoudown.mid
Eurythmics ionlywanttobewithyou.mid
Eurythmics themiracleoflove.mid
Eurythmics loveisastranger.mid
Eurythmics itsalright.mid
Eurythmics isavedtheworldtoday.mid
ZZ Top nastydogs&funkykings.mid
ZZ Top jesusjustleftchicago.mid
ZZ Top ithankyou.mid
ZZ Top stages.mid
ZZ Top roughboys.mid
ZZ Top sleepingbag.mid
ZZ Top afoolforyourstockings.mid
ZZ Top waitingonthebus.mid
ZZ Top doubleback.mid
ZZ Top pincushion.mid
ZZ Top jailhouserock.mid
ZZ Top imbadimnationwide.mid
ZZ Top gotmeunderpressure.mid
ZZ Top sharp-dressedman.mid
ZZ Top legrange.mid
Slipknot eyeless.mid
Slipknot theblisterexists.mid
Slipknot getthis.mid
Slipknot enjoythesilence.mid
Slipknot meinside.mid
Slipknot skinticket

In [5]:
# train is a list of list
# For each artist there are k number of songs
# The expected input is a batch of control sequences
for i in range(10):
    accs = meta_learner(train, test)
    print(accs)

#print(meta_learner.learner.meta_net.map_inference(input_variables[0][0:100].view(100, 1, vocabulary_size)))

[5.2058376205908825, 5.560542528638131, 5.575891550246321]


KeyboardInterrupt: 

In [ ]:
song = train[0][0]
song = song.tolist()
midi = loader.detokenize(np.array(song))
midi.write('orig.mid')

In [ ]:
midi = loader.detokenize(np.array(song[:30]))
midi.write('song.mid')

gen_seq = meta_learner.learner.learner_net.map_inference(np.array([song[:30]]))
midi = loader.detokenize(np.array(song[:30] + gen_seq[0]))
midi.write('pred.mid')

In [ ]:
torch.save(meta_learner.state_dict(), "../models/song.mod")

pickle.dump(episode, open("../models/episode", "wb"))

In [12]:
meta_learner.load_state_dict(torch.load('../models/maml_e-2_3000'))

In [ ]:
eps = pickle.load(open("../models/episode", "rb"))

In [ ]:
train = eps.support
test = eps.query

In [ ]:
support = ep

## Inference

In [135]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)



def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [196]:
eps = load_sampler_from_config("../src/config.yaml")

In [197]:
ep = eps.get_episode()

Jeff Buckley whatwillyousay.mid
Jeff Buckley corpuschristicarol.mid
Jeff Buckley demonjohn.mid
Jeff Buckley grace.mid
Jeff Buckley jenenconnaispaslafin(live).mid
Jeff Buckley haventyouheard.mid
Jeff Buckley lastgoodbye.mid
Jeff Buckley soreal.mid
Jeff Buckley witchesrave.mid
Jeff Buckley openedonce.mid
Jeff Buckley satisfiedmind.mid
Jeff Buckley nightmaresbythesea.mid
Jeff Buckley dreambrother.mid
Jeff Buckley eternallife.mid
Jeff Buckley hallelujah.mid
Santana oyemicanto.mid
Santana everybodyseverything.mid
Santana songofthewind.mid
Santana whydontyouandi.mid
Santana imgonejam.mid
Santana jingo.mid
Santana blackmagicwoman.mid
Santana smooth.mid
Santana sambapati.mid
Santana loveofmylife.mid
Santana corazónespinado.mid
Santana europa.mid
Santana mariamaria.mid
Santana shesnotthere.mid
Santana merengation.mid
Christina Aguilera whatagirlwants.mid
Christina Aguilera ladymarmaladefeat.mya&pink&lilkim.mid
Christina Aguilera genieinabottle.mid
Christina Aguilera comeonoverbaby.mid
Christina

In [198]:
pickle.dump(ep, open("ep", "wb"))

In [199]:
ep = pickle.load(open("ep", "rb"))

In [200]:
support = ep.support[1]
query = ep.query[1]

In [201]:
support

array([[4708, 4708, 4708, ...,  186, 4616, 2238],
       [4708, 4708, 4708, ..., 4616, 2509, 4611],
       [4708, 4708, 4661, ..., 4612, 2624, 4610],
       ...,
       [4708, 4708, 4708, ..., 2233, 3141, 4376],
       [4708, 4708, 4708, ..., 4637, 2624, 4613],
       [4708, 4708, 4708, ..., 4610,  572, 4634]], dtype=int32)

In [202]:
lr = 0.01
meta_learner = MetaLearner(Model, (vocabulary_size,encoding_size,vocabulary_size,lr), lr, 1)
meta_learner.load_state_dict(torch.load('../models/maml_e-3_3000'))

learner = meta_learner.learner.meta_net

In [203]:
""" Train on same song 5 times """
SEQ_LENGTH = 150 # SOS and EOS
SONG_INDEX = 0
optimizer = torch.optim.Adam(learner.parameters(), lr)

learner.train()

print_every = 1
total_epochs = 20
print_loss_total = 0
startTime = time.time()
for epoch in range(1, total_epochs+1):
    
    loss = learner(support)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print_loss_total += (loss/SEQ_LENGTH)
    
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(startTime, epoch / total_epochs),
                                     epoch, epoch / total_epochs * 100, print_loss_avg))

0m 1s (- 0m 35s) (1 5%) 3.1161
0m 4s (- 0m 38s) (2 10%) 2.7053
0m 6s (- 0m 37s) (3 15%) 2.8596
0m 8s (- 0m 35s) (4 20%) 3.0919
0m 11s (- 0m 34s) (5 25%) 2.5456
0m 13s (- 0m 32s) (6 30%) 2.0584
0m 16s (- 0m 29s) (7 35%) 1.7812
0m 18s (- 0m 27s) (8 40%) 1.5116
0m 20s (- 0m 25s) (9 45%) 1.3091
0m 23s (- 0m 23s) (10 50%) 1.1245
0m 25s (- 0m 20s) (11 55%) 0.9653
0m 27s (- 0m 18s) (12 60%) 0.8407
0m 30s (- 0m 16s) (13 65%) 0.7078
0m 32s (- 0m 13s) (14 70%) 0.5735
0m 34s (- 0m 11s) (15 75%) 0.4660
0m 37s (- 0m 9s) (16 80%) 0.3686
0m 39s (- 0m 7s) (17 85%) 0.2901
0m 41s (- 0m 4s) (18 90%) 0.2239
0m 43s (- 0m 2s) (19 95%) 0.1693
0m 45s (- 0m 0s) (20 100%) 0.1282


In [205]:
learner.eval()
song = query[1]
midi = loader.detokenize(np.array(song))
midi.write('orig.mid')

midi = loader.detokenize(np.array(song[:30]))
midi.write('song.mid')

gen_seq = learner.sample_inference(np.array([song[:30]]))
midi = loader.detokenize(np.append(song[:30], np.array(gen_seq[0])))
midi.write('pred.mid')
print(gen_seq[0])
print(song[:30])
print(np.append(song[:30], np.array(gen_seq[0])))

../src/model.py:223: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = softmax(decoder_output).data.cpu()


[2980, 4611, 2712, 4271, 664, 4613, 1091, 4610, 2499, 4610, 4220, 461, 4222, 2509, 4640, 4204, 448, 4629, 460, 4631, 2516, 4276, 470, 4614, 2509, 4631, 2518, 4610, 461, 470, 4660, 2494, 4214, 468, 4615, 668, 4613, 448, 465, 4660, 2518, 4610, 4210, 470, 4212, 446, 4631, 2518, 4277, 4214, 461, 4280, 678, 4480, 1434, 4631, 2516, 4610, 4214, 2516, 4212, 448, 4631, 448, 4612, 4472, 1429, 4629, 2513, 4210, 470, 4633, 2494, 468, 4214, 470, 4639, 446, 4631, 2516, 4610, 470, 4631, 446, 468, 4610, 2513, 4210, 448, 4631, 2518, 4610, 465, 470, 4660, 2494, 4214, 448, 4629, 2494, 467, 4610, 2513, 4210, 448, 4633, 2515, 2496, 470, 4611, 4629, 2494, 2518, 2518, 4210, 446, 4212, 446, 4631, 2516, 4710, 470, 4612, 2494, 2518, 4612, 2496, 3477, 4212, 448, 4631, 2518, 4211, 465, 4629, 2518, 4610, 2496, 4612, 4210, 461, 4212, 448, 4631, 1434, 4710, 470, 1429, 4629, 2496, 4610, 2496, 470, 4212, 446, 4212, 448, 4631, 2513, 2516, 4612, 1429, 3477, 4211, 446, 4612, 2513, 4629, 448, 4610, 2496, 470, 446, 446, 46